In [237]:
def matches_rule(part, rule):
    if isinstance(rule, str):
        return True
    
    p_value = part[rule[0]]
    thres = rule[2]
    if rule[1] == '<':
        return p_value < thres
    else:
        return p_value > thres


def check_workflows(part, wflows):
    cur_wflow = 'in'

    while True:
        if cur_wflow == 'A':
            return True
        elif cur_wflow == 'R':
            return False
        for rule in wflows[cur_wflow]:
            if matches_rule(part, rule):
                if rule == 'A':
                    return True
                elif rule == 'R':
                    return False
                elif isinstance(rule, str):
                    cur_wflow = rule
                    break
                else:
                    cur_wflow = rule[-1]
                    break

In [238]:
with open('input.txt') as f:
    workflows = {}
    parts = []
    lines = f.read().splitlines()
    idx = 0
    for i,line in enumerate(lines):
        if not line:
            idx = i
            break
        w_name, rules = line.split('{')
        rules = rules[:-1].split(',')
        w_rules = []
        for rule in rules:
            if '<' in rule or '>' in rule:
                tmp, dest = rule.split(':')
                p_type = tmp[0]
                comp = tmp[1]
                threshold = int(tmp[2:])
                w_rules.append((p_type, comp, threshold, dest))
            else:
                w_rules.append((rule))
        
        workflows[w_name] =  w_rules

    for line in lines[idx+1:]:
        part = line[1:-1].split(',')
        parts.append({'x':int(part[0][2:]), 'm':int(part[1][2:]), 'a':int(part[2][2:]), 's':int(part[3][2:])})


accepted_parts = []
for p in parts:
    if check_workflows(p, workflows):
        accepted_parts.append(p)

sum([sum(p.values()) for p in accepted_parts])

376008

In [239]:
def find_accepted_paths(wflows):
    cur_wflow = 'in'
    paths = []
    current_path = []

    def dfs(rules):
        # print(current_path,'\n')
        if not rules:
            return
        if rules[0] == 'R':
            return

        if isinstance(rules[0], tuple):
            # Add individual components of the condition to the current path
            current_path.append(rules[0])

        if rules[0] == 'A':
            # Found a leaf node with the target value, add the current path to the list of paths
            # if current_path:
            paths.append(list(current_path))
            return

        if isinstance(rules[0], str):
            if rules[0] != 'A' and rules[0] != 'R':
                for r in wflows[rules[0]]:
                    dfs([r])
        else:
            for r in rules[1:]:
                dfs([r])
            if rules[0][-1] != 'A' and rules[0][-1] != 'R':
                for r in wflows[rules[0][-1]]:
                    dfs([r])
            elif rules[0][-1] == 'A':
                dfs(['A'])
            else:
                dfs(['R'])
        
        # Backtrack: remove the individual components of the current node from the path
        if isinstance(rules[0], tuple):
            for _ in range(len(rules)):
                current_path.pop()


    dfs([cur_wflow])
    result = []

    for path in paths:
        if not path:
            path.append(('','',''))
        elif path[-1][-1] != 'A':
            path.append(('','',''))

    for path in paths:
        cur_wflow = 'in'
        res = []
        reached_A = False
        for p in path:
            idx = 0
            if reached_A:
                break
            while cur_wflow != 'A' and cur_wflow != 'R':
                q = wflows[cur_wflow][idx]
                if q == 'A':
                    reached_A = True
                    break
                if path == [('s', '<', 1351, 'px')]:
                    print(p,q)
                if isinstance(q, str):
                    if path == [('s', '<', 1351, 'px')]:    
                        print('1')
                    if q != 'A' and q != 'R':
                        cur_wflow = q
                        idx = 0
                    continue
                if p != q:
                    if path == [('s', '<', 1351, 'px')]:
                        print('2')
                    if q[1] == '<':
                        res.append((q[0], '>', q[2]-1, q[3]))
                        idx += 1
                    else:
                        res.append((q[0], '<', q[2]+1, q[3]))
                        idx += 1
                else:
                    if path == [('s', '<', 1351, 'px')]:
                        print('3')
                    res.append(p)
                    cur_wflow = p[-1]
                    idx = 0
                    break
        result.append(res)


    return result


In [241]:
result = 0
for path in find_accepted_paths(workflows):

    possible_values = {x:(1,4000) for x in 'xmas'}

    for cond in path:
        low, high = possible_values[cond[0]]
        
        if cond[1] == '>':
            possible_values[cond[0]] = (cond[2]+1,high)
        else:
            possible_values[cond[0]] = (low,cond[2]-1)

    res = 1
    for rng in possible_values.values():
        tmp = rng[1]-rng[0]+1
        res *= max(tmp,0)

    result += res

result

124078207789312